In [1]:
import os
os.environ['PYSPARK_PYTHON'] = '/nfshome/jc9033/.conda/envs/py3/bin/python3.7'

from pyspark import SparkContext
sc = SparkContext('local', 'pyspark')

from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [2]:
sc

<SparkContext master=local appName=pyspark>

# columns

In [3]:
type_independent = ['1','4','6','22','25','26','29','36','59','66','78','79','82','84','85','87','88']

In [4]:
%%time
i = "2015"
path = "data_tickets/tickets_" + i + ".csv"
rdd = sc.textFile(path).cache()
print(list(enumerate(rdd.first().split(','))))

[(0, 'Summons Number'), (1, 'Plate ID'), (2, 'Registration State'), (3, 'Plate Type'), (4, 'Issue Date'), (5, 'Violation Code'), (6, 'Vehicle Body Type'), (7, 'Vehicle Make'), (8, 'Issuing Agency'), (9, 'Street Code1'), (10, 'Street Code2'), (11, 'Street Code3'), (12, 'Vehicle Expiration Date'), (13, 'Violation Location'), (14, 'Violation Precinct'), (15, 'Issuer Precinct'), (16, 'Issuer Code'), (17, 'Issuer Command'), (18, 'Issuer Squad'), (19, 'Violation Time'), (20, 'Time First Observed'), (21, 'Violation County'), (22, 'Violation In Front Of Or Opposite'), (23, 'House Number'), (24, 'Street Name'), (25, 'Intersecting Street'), (26, 'Date First Observed'), (27, 'Law Section'), (28, 'Sub Division'), (29, 'Violation Legal Code'), (30, 'Days Parking In Effect    '), (31, 'From Hours In Effect'), (32, 'To Hours In Effect'), (33, 'Vehicle Color'), (34, 'Unregistered Vehicle?'), (35, 'Vehicle Year'), (36, 'Meter Number'), (37, 'Feet From Curb'), (38, 'Violation Post Code'), (39, 'Violatio

# define functions & test

In [5]:
def mapper1(partitionID, rows):
    
    import pandas as pd
    dict1 = pd.read_csv('output_tickets_StreetName_TaxiZone/tickets_StreetName_TaxiZone.csv').fillna(0)\
              .set_index('StreetName').to_dict()
    
    if partitionID==0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    
    for fields in reader:
        if fields[5] not in type_independent:
            if fields[24]:
                try:
                    zone = dict1['TaxiZone'][fields[24]]
                    yield((fields[4],zone),1)
                except:
                    continue
                
def mapper2(partition):
    for element in partition:
        yield(element[0][0], element[0][1], element[1])

In [6]:
i = "2015"
path = "data_tickets/tickets_" + i + ".csv"
rdd = sc.textFile(path)\
         .mapPartitionsWithIndex(mapper1)\
         .groupByKey().mapValues(sum)\
         .mapPartitions(mapper2)

In [7]:
%%time
rdd.take(5)

CPU times: user 24 ms, sys: 4 ms, total: 28 ms
Wall time: 2min 29s


[('11/20/2014', 38, 267),
 ('06/12/2015', 229, 176),
 ('05/11/2015', 163, 5),
 ('05/19/2015', 117, 96),
 ('07/01/2014', 237, 1144)]

# Cleaning & Integrating

In [8]:
%%time
for i in range(2014,2020):
    print(i)
    path = "data_tickets/tickets_" + str(i) + ".csv"
    if i == 2014:
        rdd = sc.textFile(path)\
                 .mapPartitionsWithIndex(mapper1)\
                 .groupByKey().mapValues(sum)\
                 .mapPartitions(mapper2)
    else:
        temp = sc.textFile(path)\
                 .mapPartitionsWithIndex(mapper1)\
                 .groupByKey().mapValues(sum)\
                 .mapPartitions(mapper2)
        rdd = rdd.union(temp)
    rdd.take(5)

2014
2015
2016
2017
2018
2019
CPU times: user 168 ms, sys: 24 ms, total: 192 ms
Wall time: 10min 35s


### Output CSV

In [9]:
%%time
from pyspark import SparkContext
df = spark.createDataFrame(rdd, ['date', 'zone', 'count'])

CPU times: user 280 ms, sys: 84 ms, total: 364 ms
Wall time: 3.13 s


In [14]:
df.show()

+----------+----+-----+
|      date|zone|count|
+----------+----+-----+
|08/13/2000|  60|    1|
|08/18/2000| 135|    1|
|10/02/2000| 141|    1|
|10/04/2000| 165|    1|
|10/15/2000| 248|    1|
|10/21/2000| 156|    1|
|11/19/2000|  99|    1|
|02/28/2001| 238|    1|
|03/30/2001| 212|    1|
|12/27/2001|  22|    1|
|01/01/2004|  17|    1|
|12/22/2005|  42|    1|
|05/12/2010| 161|    1|
|05/13/2010| 169|    1|
|06/09/2011|  40|    1|
|08/22/2011| 197|    1|
|04/11/2012|  84|    1|
|05/04/2012| 236|    1|
|05/11/2012| 128|    1|
|07/24/2012| 150|    1|
+----------+----+-----+
only showing top 20 rows



In [15]:
%%time
df.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('output_tickets_zone_filterType')

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 24 s


###  Check file

In [16]:
filename = ! ls output_tickets_zone_filterType | grep .csv

In [17]:
import pandas as pd
pddf = pd.read_csv('output_tickets_zone_filterType/'+filename[0]).fillna(0)
print(pddf.shape)

(561439, 3)


In [18]:
pddf.head()

,date,zone,count
0,08/13/2000,60,1
1,08/18/2000,135,1
2,10/02/2000,141,1
3,10/04/2000,165,1
4,10/15/2000,248,1


In [19]:
pddf.tail()

,date,zone,count
561434,01/02/2018,245,1
561435,01/28/2019,43,16
561436,01/16/2019,139,1
561437,01/28/2019,113,26
561438,12/30/2018,261,4
